# Import Target, Related and Item Meta Dataset
* This notebook has the following processes:
    * Create IAM role
    * Create a dataset group
    * Create a schema for a dataset
    * Create the dataset
    * Attach the dataset to the dataset group
    * Create a dataset import job
    
    
* **About 10 mins may be elapsed**


In [1]:
import boto3
from time import sleep
import os
import pandas as pd
import json
import time
import pprint
import numpy as np

## Parameters

In [2]:
%store -r

In [41]:
DATASET_FREQUENCY = "W" 
TIMESTAMP_FORMAT = "yyyy-MM-dd"

project = 'WalmartKaggleWithThreeDatasets'
suffix = str(np.random.uniform())[4:9]
target_suffix = '_target'
related_suffix = '_related'
item_meta_suffix = '_ItemM'

target_datasetName= project+'DS' + target_suffix + suffix
item_meta_dataset_name= project+'DS' + item_meta_suffix + suffix
related_dataset_Name= project+'DS' + related_suffix + suffix
item_datasetGroupName= project +'DSG'+ item_meta_suffix + suffix

In [4]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
     data = json.load(notebook_info)
     resource_arn = data['ResourceArn']
     region = resource_arn.split(':')[3]
print(region)

us-east-2


In [5]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')
forecast_query = session.client(service_name='forecastquery')

## Create role
**Make sure that a role for SageMaker notebook instance has these policies attached such as AmazonSageMakerFullAccess, AmazonS3FullAccess, AmazonForecastFullAccess, IAMFullAccess**

In [7]:
iam = boto3.client("iam")

# Put the role name
role_name = "ForecastRoleWalmart" + suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "forecast.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like tåo use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/AmazonForecastFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::057716757052:role/ForecastRoleWalmart35988


## Create DatasetGroup

In [8]:
# Create the DatasetGroup
create_dataset_group_response = forecast.create_dataset_group(
      DatasetGroupName=item_datasetGroupName,
      Domain="CUSTOM",
     )
item_meta_datasetGroupArn = create_dataset_group_response['DatasetGroupArn']

In [9]:
forecast.describe_dataset_group(DatasetGroupArn=item_meta_datasetGroupArn)

{'DatasetGroupName': 'WalmartKaggleWithThreeDatasetsDSG_ItemM35988',
 'DatasetGroupArn': 'arn:aws:forecast:us-east-2:057716757052:dataset-group/WalmartKaggleWithThreeDatasetsDSG_ItemM35988',
 'DatasetArns': [],
 'Domain': 'CUSTOM',
 'Status': 'ACTIVE',
 'CreationTime': datetime.datetime(2020, 4, 5, 6, 18, 41, 965000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 5, 6, 18, 41, 965000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'd1e1a5ef-3abe-4bf1-b314-195c817b7c2e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 05 Apr 2020 06:18:49 GMT',
   'x-amzn-requestid': 'd1e1a5ef-3abe-4bf1-b314-195c817b7c2e',
   'content-length': '311',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Create schema for target data

In [10]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
target_schema ={
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"target_value",
         "AttributeType":"float"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      }
   ]
}

## Create Target Time Sereis Dataset

In [11]:
response=forecast.create_dataset(
                    Domain="CUSTOM",
                    DatasetType='TARGET_TIME_SERIES',
                    DatasetName=target_datasetName,
                    DataFrequency=DATASET_FREQUENCY, 
                    Schema = target_schema
)

In [13]:
target_second_datasetArn = response['DatasetArn']
forecast.describe_dataset(DatasetArn=target_second_datasetArn)

{'DatasetArn': 'arn:aws:forecast:us-east-2:057716757052:dataset/WalmartKaggleWithThreeDatasetsDS_target35988',
 'DatasetName': 'WalmartKaggleWithThreeDatasetsDS_target35988',
 'Domain': 'CUSTOM',
 'DatasetType': 'TARGET_TIME_SERIES',
 'DataFrequency': 'W',
 'Schema': {'Attributes': [{'AttributeName': 'timestamp',
    'AttributeType': 'timestamp'},
   {'AttributeName': 'target_value', 'AttributeType': 'float'},
   {'AttributeName': 'item_id', 'AttributeType': 'string'}]},
 'EncryptionConfig': {},
 'Status': 'ACTIVE',
 'CreationTime': datetime.datetime(2020, 4, 5, 6, 18, 55, 721000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 5, 6, 18, 55, 721000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '8ed27c6a-dc8e-4e41-83f6-398efa35b3ed',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 05 Apr 2020 06:21:20 GMT',
   'x-amzn-requestid': '8ed27c6a-dc8e-4e41-83f6-398efa35b3ed',
   'content-length': '549',
 

## Create schema for related data

In [15]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
related_schema ={
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"Temperature",
         "AttributeType":"float"
      },
      {
         "AttributeName":"Fuel_Price",
         "AttributeType":"float"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      }       
   ]
}

## Create Related Time Sereis Dataset

In [16]:
response=forecast.create_dataset(
                    Domain="CUSTOM",
                    DatasetType='RELATED_TIME_SERIES',
                    DatasetName=related_dataset_Name,
                    DataFrequency=DATASET_FREQUENCY, 
                    Schema = related_schema
)

In [17]:
related_datasetArn = response['DatasetArn']
forecast.describe_dataset(DatasetArn=related_datasetArn)

{'DatasetArn': 'arn:aws:forecast:us-east-2:057716757052:dataset/WalmartKaggleWithThreeDatasetsDS_related35988',
 'DatasetName': 'WalmartKaggleWithThreeDatasetsDS_related35988',
 'Domain': 'CUSTOM',
 'DatasetType': 'RELATED_TIME_SERIES',
 'DataFrequency': 'W',
 'Schema': {'Attributes': [{'AttributeName': 'timestamp',
    'AttributeType': 'timestamp'},
   {'AttributeName': 'Temperature', 'AttributeType': 'float'},
   {'AttributeName': 'Fuel_Price', 'AttributeType': 'float'},
   {'AttributeName': 'item_id', 'AttributeType': 'string'}]},
 'EncryptionConfig': {},
 'Status': 'ACTIVE',
 'CreationTime': datetime.datetime(2020, 4, 5, 6, 21, 28, 676000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 5, 6, 21, 28, 676000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '5d55938b-9956-4d84-84b0-0daea6b2620b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 05 Apr 2020 06:21:32 GMT',
   'x-amzn-requestid': '5d55

## Create schema for Item Meta data

In [18]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
item_meta_schema ={
   "Attributes":[
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      },       
      {
         "AttributeName":"StoreType",
         "AttributeType":"string"
      }       
   ]
}

## Create Item-Meta Dataset

In [19]:
response=forecast.create_dataset(
                    Domain="CUSTOM",
                    DatasetType='ITEM_METADATA',
                    DatasetName=item_meta_dataset_name,
                    DataFrequency=DATASET_FREQUENCY, 
                    Schema = item_meta_schema
)

In [20]:
item_meta_datasetArn = response['DatasetArn']
forecast.describe_dataset(DatasetArn=item_meta_datasetArn)

{'DatasetArn': 'arn:aws:forecast:us-east-2:057716757052:dataset/WalmartKaggleWithThreeDatasetsDS_ItemM35988',
 'DatasetName': 'WalmartKaggleWithThreeDatasetsDS_ItemM35988',
 'Domain': 'CUSTOM',
 'DatasetType': 'ITEM_METADATA',
 'Schema': {'Attributes': [{'AttributeName': 'item_id',
    'AttributeType': 'string'},
   {'AttributeName': 'StoreType', 'AttributeType': 'string'}]},
 'EncryptionConfig': {},
 'Status': 'ACTIVE',
 'CreationTime': datetime.datetime(2020, 4, 5, 6, 21, 46, 13000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 5, 6, 21, 46, 13000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'f47f33fc-e6f6-4a3f-a7c0-e11df74a9db7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 05 Apr 2020 06:22:58 GMT',
   'x-amzn-requestid': 'f47f33fc-e6f6-4a3f-a7c0-e11df74a9db7',
   'content-length': '462',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Attach the target, related and item_meta dataset to the DatasetGroup

In [21]:
# Attach the target dataset and related data set  to the Dataset Group:
forecast.update_dataset_group(
    DatasetGroupArn=item_meta_datasetGroupArn, 
    DatasetArns=[target_second_datasetArn,
                 related_datasetArn,
                 item_meta_datasetArn])

{'ResponseMetadata': {'RequestId': '0812c913-2a4b-404d-b2c2-1cd8efcffeb0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 05 Apr 2020 06:23:05 GMT',
   'x-amzn-requestid': '0812c913-2a4b-404d-b2c2-1cd8efcffeb0',
   'content-length': '2',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Create a bucket

In [22]:
import boto3
import sagemaker

s3_resource = boto3.resource('s3')
s3 = boto3.client('s3')

# if you want, replace with a name of your S3 bucket
bucket_name = sagemaker.Session().default_bucket()  

if s3_resource.Bucket(bucket_name).creation_date is None:
    # bucket is not existing 
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})    
else: 
    # Bucket exists
    print("bucket name is ", bucket_name)
    

bucket name is  sagemaker-us-east-2-057716757052


### Upload three data to S3

In [30]:
# Upload Target File under a bucket folder
bucket_folder = project
s3_file_path = bucket_folder + "/" + target_time_series_filename

boto3.Session().resource('s3').Bucket(bucket_name).Object(s3_file_path).upload_file(target_time_series_path)
target_s3DataPath = "s3://"+bucket_name + "/" + s3_file_path

In [31]:
# Upload Related File under a bucket folder
bucket_folder = project
s3_file_path = bucket_folder + "/" + related_time_series_filename

boto3.Session().resource('s3').Bucket(bucket_name).Object(s3_file_path).upload_file(related_time_series_path)
related_s3DataPath = "s3://"+bucket_name + "/" + s3_file_path

In [32]:
# Upload Item Meta File under a bucket folder
bucket_folder = project
s3_file_path = bucket_folder + "/" + store_meta_filename

boto3.Session().resource('s3').Bucket(bucket_name).Object(s3_file_path).upload_file(store_meta_path)
item_meta_s3DataPath = "s3://"+bucket_name + "/" + s3_file_path

## Create dataset_import_job used to download dataset from S3

In [26]:
# Target Import Job
datasetImportJobName = 'DSIMPORT_JOB_TARGET_WALMART' + suffix
ds_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=datasetImportJobName,
                                                          DatasetArn=target_second_datasetArn,
                                                          DataSource= {
                                                              "S3Config" : {
                                                                 "Path":target_s3DataPath,
                                                                 "RoleArn": role_arn
                                                              } 
                                                          },
                                                          TimestampFormat=TIMESTAMP_FORMAT
                                                         )

In [27]:
ds_target_second_import_job_arn=ds_import_job_response['DatasetImportJobArn']
print(ds_target_second_import_job_arn)

arn:aws:forecast:us-east-2:057716757052:dataset-import-job/WalmartKaggleWithThreeDatasetsDS_target35988/DSIMPORT_JOB_TARGET_WALMART35988


In [33]:
# Related Import Job
datasetImportJobName = 'DSIMPORT_JOB_RELATED_WALMART' + related_suffix + suffix
ds_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=datasetImportJobName,
                                                          DatasetArn=related_datasetArn,
                                                          DataSource= {
                                                              "S3Config" : {
                                                                 "Path":related_s3DataPath,
                                                                 "RoleArn": role_arn
                                                              } 
                                                          },
                                                          TimestampFormat=TIMESTAMP_FORMAT
                                                         )

In [34]:
ds_related_import_job_arn=ds_import_job_response['DatasetImportJobArn']
print(ds_related_import_job_arn)

arn:aws:forecast:us-east-2:057716757052:dataset-import-job/WalmartKaggleWithThreeDatasetsDS_related35988/DSIMPORT_JOB_RELATED_WALMART_related35988


In [35]:
# Finally we can call import the dataset
datasetImportJobName = 'DSIMPORT_JOB_RELATED_WALMART' + related_suffix + suffix
ds_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=datasetImportJobName,
                                                          DatasetArn=item_meta_datasetArn,
                                                          DataSource= {
                                                              "S3Config" : {
                                                                 "Path":item_meta_s3DataPath,
                                                                 "RoleArn": role_arn
                                                              } 
                                                          },
                                                          TimestampFormat=TIMESTAMP_FORMAT
                                                         )

In [36]:
ds_itme_meta_import_job_arn=ds_import_job_response['DatasetImportJobArn']
print(ds_itme_meta_import_job_arn)

arn:aws:forecast:us-east-2:057716757052:dataset-import-job/WalmartKaggleWithThreeDatasetsDS_ItemM35988/DSIMPORT_JOB_RELATED_WALMART_related35988


In [37]:
%%time

while True:
    dataTargetImportStatus = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_target_second_import_job_arn)['Status']
    print("Target: ", dataTargetImportStatus)
    dataRelatedImportStatus = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_related_import_job_arn)['Status']
    print("Related: ", dataRelatedImportStatus)
    dataItemMetaImportStatus = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_related_import_job_arn)['Status']
    print("Item Metadata: ", dataItemMetaImportStatus)    
    if dataTargetImportStatus != 'ACTIVE' and dataTargetImportStatus != 'CREATE_FAILED':
        sleep(30)
    elif dataRelatedImportStatus != 'ACTIVE' and dataRelatedImportStatus != 'CREATE_FAILED':
        sleep(30)
    elif dataItemMetaImportStatus != 'ACTIVE' and dataItemMetaImportStatus != 'CREATE_FAILED':
        sleep(30)    
    else:
        break

Target:  CREATE_IN_PROGRESS
Related:  CREATE_PENDING
Item Metadata:  CREATE_PENDING
Target:  CREATE_IN_PROGRESS
Related:  CREATE_IN_PROGRESS
Item Metadata:  CREATE_IN_PROGRESS
Target:  CREATE_IN_PROGRESS
Related:  CREATE_IN_PROGRESS
Item Metadata:  CREATE_IN_PROGRESS
Target:  CREATE_IN_PROGRESS
Related:  CREATE_IN_PROGRESS
Item Metadata:  CREATE_IN_PROGRESS
Target:  ACTIVE
Related:  CREATE_IN_PROGRESS
Item Metadata:  CREATE_IN_PROGRESS
Target:  ACTIVE
Related:  CREATE_IN_PROGRESS
Item Metadata:  CREATE_IN_PROGRESS
Target:  ACTIVE
Related:  CREATE_IN_PROGRESS
Item Metadata:  CREATE_IN_PROGRESS
Target:  ACTIVE
Related:  CREATE_IN_PROGRESS
Item Metadata:  CREATE_IN_PROGRESS
Target:  ACTIVE
Related:  CREATE_IN_PROGRESS
Item Metadata:  CREATE_IN_PROGRESS
Target:  ACTIVE
Related:  CREATE_IN_PROGRESS
Item Metadata:  CREATE_IN_PROGRESS
Target:  ACTIVE
Related:  CREATE_IN_PROGRESS
Item Metadata:  CREATE_IN_PROGRESS
Target:  ACTIVE
Related:  CREATE_IN_PROGRESS
Item Metadata:  CREATE_IN_PROGRESS


In [43]:
%store project
%store region
%store bucket_name
%store bucket_folder
%store role_arn
%store role_name
%store suffix
%store target_suffix
%store item_meta_suffix
%store related_suffix

%store item_meta_datasetGroupArn
%store target_second_datasetArn
%store related_datasetArn
%store item_meta_datasetArn
%store ds_target_second_import_job_arn
%store ds_related_import_job_arn
%store ds_itme_meta_import_job_arn




Stored 'project' (str)
Stored 'region' (str)
Stored 'bucket_name' (str)
Stored 'bucket_folder' (str)
Stored 'role_arn' (str)
Stored 'role_name' (str)
Stored 'suffix' (str)
Stored 'target_suffix' (str)
Stored 'item_meta_suffix' (str)
Stored 'related_suffix' (str)
Stored 'item_meta_datasetGroupArn' (str)
Stored 'target_second_datasetArn' (str)
Stored 'related_datasetArn' (str)
Stored 'item_meta_datasetArn' (str)
Stored 'ds_target_second_import_job_arn' (str)
Stored 'ds_related_import_job_arn' (str)
Stored 'ds_itme_meta_import_job_arn' (str)
